<img src="../Images/DSC_Logo.png" style="width: 400px;">

# Interactive Plots: Plotly

To create interactive visualisations in Python, we focus on plotly. This library enables the creation of web-ready plots that increase user engagement through features. Plotly automatically enables zooming (zoom in on specific areas of the plot), panning (click and drag the plot area), and hovering (detailed information about a specific data point is shown). For web deployment, plotly is often paired with Dash to build interactive applications. `fig.write_html` can be used to save an interactive plotly visualization as an HTML file. Alternative libraries such as bokeh exist to create web-ready interactive plots.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
import plotly.express as px
import plotly.graph_objects as go

## 1. Zooming, Panning, and Hovering Elements

### **Example 1 [continued from B2]: Antarctic CO2 concentrations**

Let's create an interactive plot of the time series data showing CO2 concentrations in Antarctica, allowing for the exploration and examination of specific time periods  Implement interactive zooming and panning functionalities so users can seamlessly navigate through the time series data.

In [ ]:
# Load dataset as before
path = '../Datasets/antarctica/antarctica2015co2composite-noaa.txt' 
co2_antarctica = pd.read_csv(path, 
                             skiprows=142,
                             sep="\t")

Importing plotly as `plotly.express` is for quick and straightforward visualizations.

Create a Plotly Express line plot:

In [ ]:
fig = px.line(
    co2_antarctica, 
    x='age_gas_calBP', 
    y='co2_ppm',
    title='Antarctic Atmospheric CO2 Concentrations',
    labels={'age_gas_calBP': 'Years Before Present (BP)', 'co2_ppm': 'CO2 Concentration (ppm)'},
    template='plotly_white'
    # markers=True
)

# Update the trace to specify the color
fig.update_traces(line=dict(color='blue'))

# Update the layout for a centered title and custom dimensions
fig.update_layout(
    title_x=0.5,  # Center the title
    width=1000,   # Width of the plot
    height=600    # Height of the plot
)

# Invert the x-axis
fig.update_xaxes(autorange='reversed')

# Show the plot
fig.show()

Create a Plotly Express scatter plot:

In [ ]:
fig = px.scatter(
    co2_antarctica, 
    x='age_gas_calBP', 
    y='co2_ppm',
    title='Antarctic Atmospheric CO2 Concentrations',
    labels={'age_gas_calBP': 'Years Before Present (BP)', 'co2_ppm': 'CO2 Concentration (ppm)'},
    template='plotly_white',
)

# Set the size and color of the markers after creating the plot
fig.update_traces(marker=dict(size=3, color='blue')) 

# Update the layout for a centered title and custom dimensions
fig.update_layout(
    title_x=0.5,  # Center the title
    width=1000,   # Width of the plot
    height=600    # Height of the plot
)

# Invert the x-axis
fig.update_xaxes(autorange='reversed')

# Show the plot
fig.show()

Importing plotly as `plotly.graph_objects` is required when detailed control over plot elements is needed. Below we see that `add_trace` is set before the definition of a scatter plot. In plotly, a trace is a data representation or a visual layer in a plot that corresponds to a specific dataset or series of data. Each trace can be thought of as one individual element of the overall visualization. Traces are the building blocks of plotly figures, and it is possible to have multiple traces in a single figure to represent different datasets or different aspects of the data. 

To reproduce the time series plot from notebook B2, two traces are required: a `go.Scatter` plot that is set up with lines and markers, and a filled area also set up with a `go.Scatter` plot in the first place. 

`update_` is generally used to customize the overall layout and properties of the figure, including titles, axis labels, and template styles. For example, the `update_xaxes` method is used to reverse the axis order to show the timeline as years before present from left to right.

In [ ]:
# Create a plotly go figure
fig = go.Figure()

# Add the CO2 concentration line plot with markers
fig.add_trace(go.Scatter(
    x=co2_antarctica['age_gas_calBP'],
    y=co2_antarctica['co2_ppm'],
    mode='lines+markers',
    marker=dict(size=3),
    line=dict(width=1),
    name='CO2 Concentration (ppm)',
    line_shape='linear',
    marker_color='blue',
    showlegend=False
))

# Add the filled area for uncertainty (± 1 Standard Deviation)
fig.add_trace(go.Scatter(
    x=co2_antarctica['age_gas_calBP'].tolist() + co2_antarctica['age_gas_calBP'][::-1].tolist(),
    y=(co2_antarctica['co2_ppm'] + co2_antarctica['co2_1s_ppm']).tolist() + (co2_antarctica['co2_ppm'] - co2_antarctica['co2_1s_ppm'])[::-1].tolist(),
    fill='toself',
    fillcolor='rgba(0, 0, 255, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo='skip',
    showlegend=False,
    name='± 1 Standard Deviation'
))

# Update the layout for a centered title and custom dimensions
fig.update_layout(
    title_x=0.5,  # Center the title
    width=1000,   # Width of the plot
    height=600    # Height of the plot
)

# Invert the x-axis
fig.update_xaxes(autorange="reversed")

# Update layout for title and labels
fig.update_layout(
    title={
        'text': 'Antarctic Atmospheric CO2 Concentrations',
        'x': 0.5,  # Center the title
        'xanchor': 'center'
    },
    xaxis_title='Years Before Present (BP)',
    yaxis_title='CO2 Concentration (ppm)',
    template='plotly_white'
)

# Show the plot
fig.show()

### **Example 2 [continued from B2]: Chemical concentrations in volcanic tephra**

In [ ]:
# Load dataset as before
path = '../Datasets/Smith_glass_post_NYT_data.xlsx'
traces = pd.read_excel(path, sheet_name=1)
majors = pd.read_excel(path)

# Define the color mapping for each epoch as before
color_map = {'one':'red', 'two':'blue', 'three':'purple', 'three-b':'orange'}

**Exercise:** Create an interactive scatter plot to visualize the relationship between Zr and Th concentrations, with color-coding for different epochs using plotly. Assign colors to epochs by first using `color='Epoch'` and then specifying `color_discrete_map=color_map` to define color mappings inside the `go.Scatter` plot. Optimize the layout.

## 2. Subplots in Plotly

`fig.add_trace` is a method used in plotly to add individual traces (plots) to a figure object. It is essential in subplots when manual control of the layout and content of each subplot in a grid defined by `make_subplots` is required.

In [ ]:
from plotly.subplots import make_subplots

# Define oxide columns to plot in individual subplots
oxide_columns = ['SIO2', 'CAO', 'FEO', 'MGO', 'P2O5', 'Cl']

# Create a subplot grid with 2 rows and 3 columns
fig = make_subplots(rows=2, cols=3, subplot_titles=oxide_columns)

# Add traces for each oxide by specifying row and column
fig.add_trace(
    go.Histogram(
        x=majors[oxide_columns[0]],
        marker=dict(color='blue', line=dict(color='black', width=1))
    ),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(
        x=majors[oxide_columns[1]],
        marker=dict(color='green', line=dict(color='black', width=1))
    ),
    row=1, col=2
)

fig.add_trace(
    go.Histogram(
        x=majors[oxide_columns[2]],
        marker=dict(color='purple', line=dict(color='black', width=1))
    ),
    row=1, col=3
)

fig.add_trace(
    go.Histogram(
        x=majors[oxide_columns[3]],
        marker=dict(color='orange', line=dict(color='black', width=1))
    ),
    row=2, col=1
)

fig.add_trace(
    go.Histogram(
        x=majors[oxide_columns[4]],
        marker=dict(color='red', line=dict(color='black', width=1))
    ),
    row=2, col=2
)

fig.add_trace(
    go.Histogram(
        x=majors[oxide_columns[5]],
        marker=dict(color='cyan', line=dict(color='black', width=1))
    ),
    row=2, col=3
)

# Update axis labels for each subplot
for i, column in enumerate(oxide_columns, start=1):
    row = (i - 1) // 3 + 1
    col = (i - 1) % 3 + 1
    fig.update_xaxes(title_text=f"{column} [ppm]", row=row, col=col)
    fig.update_yaxes(title_text="Counts", row=row, col=col)

# Update the layout
fig.update_layout(
    height=600,  # Adjust the height if necessary
    title_text="Histograms of Major Element Concentrations",
    showlegend=False,
    template='plotly_white'
)

# Update the layout for a centered title and custom dimensions
fig.update_layout(
    title_x=0.5,  # Center the title
    width=1200,   # Width of the plot
    height=500    # Height of the plot
)

# Show plot
fig.show()

## 2. Interactive Maps

### **Example 3 [continued from B3]: Köppen-Geiger maps for 1901–2099**

To create an interactive map with plotly using the Köppen-Geiger data, loading the dataset with rasterio and masking the dataset to exclude missing data for the oceans is similar as in the static plot. The `go.Heatmap` function, however, requires the color definition to be formatted differently.

In [ ]:
import rasterio

# Prepare custom colors
file_path = '../Datasets/koppen_geiger/koppen_table.csv'
colors = pd.read_csv(file_path)
# Create a colorscale mapping
colorscale = [[i/(len(colors)-1), f'rgba({row["Red"]}, {row["Green"]}, {row["Blue"]}, 1)'] for i, row in colors.iterrows()]

# Load raster data with rasterio
file_path = '../Datasets/koppen_geiger/koppen_geiger_1p0_2071_2099_ssp245.tif'
with rasterio.open(file_path) as src:
    data = src.read(1)  # Read the first band

# Mask the zeros in the data
masked_data = np.ma.masked_equal(data, 0)

Creating a pandas dataframe from the masked raster data is necessary for handling hover text in the plotly plot:

In [ ]:
# Dataframe for raster data
df = pd.DataFrame(masked_data)

`value_to_symbol` mapping is used for the interactive visualization to translate raster values into their corresponding Köppen-Geiger climate class symbols for hover text:

In [ ]:
# Create a mapping of raster values to their corresponding symbols
value_to_symbol = {i + 1: symbol for i, symbol in enumerate(colors['Symbol'].tolist())}

# Create hover text
hover_text = df.applymap(lambda value: value_to_symbol.get(int(value), 'No Data') if not np.isnan(value) else 'No Data')

A `go.Figure` is initialized, and a `go.Heatmap` trace is added to visualize the climate classifications. Hover text is incorporated to display corresponding climate class symbols when users interact with the map.

Plotly does not directly support Cartopy. However, contours and features using plotly's built-in geographical capabilities could be overlayed (not done here). Let's explore that in the next example.

In [ ]:
# Create go.Figure
fig = go.Figure(data=go.Heatmap(
    z=df,
    colorscale=colorscale,  # Use the colorscale
    zmin=1,  # Minimum value for the colormap
    zmax=30,  # Maximum value for the colormap
    text=hover_text.values,  # Set hover text to display class symbols
    hoverinfo='text',  # Show text on hover
    showscale=False  # Remove the color bar
))

# Update layout
fig.update_layout(
    title="Köppen-Geiger Climate Classifications (2071-2099)",
    title_x=0.5,  # Center the title
    width=1200,   # Width of the plot
    height=800,  # Height of the plot
    template='plotly_white'
)

# Reverse the y-axis to correct the orientation
fig.update_yaxes(autorange='reversed')

# Show the plot
fig.show()

### **Example 4 [continued from B3]: POLARSTERN cruise PS141 master track**

To convert the static polar map into an interactive map using plotly, we'll employ `go.Scattergeo` feature. This will allow us to plot the track on a map with interactive zooming and panning features. Plotly automatically displays essential geographical elements such as land areas and ocean regions without requiring manual overlaying of these features with `go.Scattergeo`. In contrast, with raster data any desired geographical context must be added explicitly. 

What movements can be observed in the Polarstern Expedition 141's path along the Antarctic coast?

In [ ]:
# Load dataset
path = '../Datasets/PS141_mastertrack.tab' 
track = pd.read_csv(path, skiprows=21, sep="\t")

Let's first create a plotly figure that displays the track of the Polarstern expedition on a flat geometric representation. Here with plotly express:

In [ ]:
# plotly express with px.line_geo
fig = px.line_geo(track,
                  lon='Longitude',
                  lat='Latitude',
                  title='Polarstern Expedition 141 Track')

# Update the line color to red
fig.update_traces(line=dict(color='red', width=2))

# Update layout for specific dimensions
fig.update_layout(
    title_x=0.5,  # Center the title
    width=1200,   # Width of the plot
    height=800    # Height of the plot
)

# Show the plot
fig.show()

And here with plotly go:

In [ ]:
fig = go.Figure()

# Add the track using Scattergeo
fig.add_trace(go.Scattergeo(
    lon=track['Longitude'],
    lat=track['Latitude'],
    mode='lines',  # Connect the points with lines
    line=dict(color='red', width=2),  # Line color and width
    name='Polarstern Expedition 141 Track'
))

# Update layout without the scope property
fig.update_layout(
    title='Polarstern Expedition 141 Track',
    title_x=0.5,  # Center the title
    width=1200,   # Width of the plot
    height=800   # Height of the plot
)

# Show the plot
fig.show()

Use a stereographic projection to to focus on the Antarctic. The `geo` configuration controls how the geographical map is presented in terms of the projection style, color schemes, and the extent of the visible area represented on the map. Again, here are solutions for both plotly express and plotly go:

In [ ]:
# plotly express with px.line_geo
fig = px.line_geo(
    track,
    lon='Longitude',
    lat='Latitude',
    title='Polarstern Expedition 141 Track'
)

# Update the line color to red and set line width
fig.update_traces(line=dict(color='red', width=2))

# Update layout without the scope property
fig.update_layout(
    title_x=0.5,  # Center the title
    width=1000,   # Width of the plot
    height=800,   # Height of the plot
    geo=dict(
        projection_type='stereographic',  # Polar projection to focus on the Antarctic
        showland=True,
        landcolor='gray',                 # Color for land
        oceancolor='lightblue',           # Color for ocean
        lonaxis=dict(range=[-180, 180]),  # Longitude range
        lataxis=dict(range=[-90, -22.9])   # Latitude range for the Antarctic region
    )
)

# Show the plot
fig.show()

In [ ]:
fig = go.Figure()

# Add the track using Scattergeo
fig.add_trace(go.Scattergeo(
    lon=track['Longitude'],
    lat=track['Latitude'],
    mode='lines',  # Connect the points with lines
    line=dict(color='red', width=2),  # Line color and width
    name='Polarstern Expedition 141 Track'
))

# Update layout without the scope property
fig.update_layout(
    title='Polarstern Expedition 141 Track',
    title_x=0.5,  # Center the title
    width=1000,   # Width of the plot
    height=800,   # Height of the plot
    geo=dict(
        projection_type='stereographic',  # Polar projection to focus on the Antarctic
        showland=True,
        landcolor='gray',                 # Color for land
        oceancolor='lightblue',           # Color for ocean
        lonaxis=dict(range=[-180, 180]),  # Longitude range
        lataxis=dict(range=[-90, -22.9])   # Latitude range for the Antarctic region
    )
)

# Show the plot
fig.show()

### **Example 5 [continued from B2/B3]: Antarctic CO2 concentrations - stations**

**Exercise:** Plot the Antarctic ice core stations using plotly express with `px.scatter_geo` instead of `px.line_geo` and use stereographic projection.

In [ ]:
# Site data from metadata 
site = ["Dome C", "Vostok", "Siple Dome", "TALDICE", "EDML"]
elevation = [3233, 3488, 621, 2315, 2892]
latitude = [-75.1, -78.47, -81.65, -72.8166, -75.0]
longitude = [123.4, 106.8, -148.82, 159.1833, 0.07]

# Create dataFrame
antarctica = pd.DataFrame({
    "Site": site,
    "Elevation masl": elevation,
    "Lat": latitude,
    "Lon": longitude
})

## 3. Dynamic Selection

Interactive plots allow users to select the data they wish to visualise using elements such as drop-down menus, sliders or buttons. This interactive approach provides flexibility, allowing different datasets or plot configurations to be displayed in response to user input.

### **Example 6 [continued from B2]: Temperature anomalies**

In the example below, we can select from several regional datasets provided by NOAA to visualise temperature anomalies. The plot includes a drop-down menu for switching between regions, with accompanying bar graphs that update to show temperature anomalies over time for the selected region.

We first define regions and their corresponding file paths to choose from in the interactive plot:

In [ ]:
regions = {
    'Africa': '../Datasets/noaa_temperature_anomalies/Africa.csv',
    'Antarctic': '../Datasets/noaa_temperature_anomalies/Antarctic.csv',
    'Arctic': '../Datasets/noaa_temperature_anomalies/Arctic.csv',
    'Asia': '../Datasets/noaa_temperature_anomalies/Asia.csv',
    'Atlantic_MDR': '../Datasets/noaa_temperature_anomalies/Atlantic_MDR.csv',
    'Caribbean_Islands': '../Datasets/noaa_temperature_anomalies/Caribbean_Islands.csv',
    'East_N_Pacific': '../Datasets/noaa_temperature_anomalies/East_N_Pacific.csv',
    'Europe': '../Datasets/noaa_temperature_anomalies/Europe.csv',
    'Gulf_of_Mexico': '../Datasets/noaa_temperature_anomalies/Gulf_of_Mexico.csv',
    'Hawaiian_Region': '../Datasets/noaa_temperature_anomalies/Hawaiian_Region.csv',
    'North_America': '../Datasets/noaa_temperature_anomalies/North_America.csv',
    'Oceania': '../Datasets/noaa_temperature_anomalies/Oceania.csv',
    'South_America': '../Datasets/noaa_temperature_anomalies/South_America.csv'
}

We want to plot the annual means as in notebook B2.

In [ ]:
# Create an empty DataFrame to hold all the annual means
annual_means = pd.DataFrame()

# Load each region's data and compute the annual mean anomalies
for region, path in regions.items():
    anomalies = pd.read_csv(path, skiprows=4)
    
    # Convert the 'Date' column to datetime format
    anomalies['Date'] = pd.to_datetime(anomalies['Date'].astype(str), format='%Y%m')
    anomalies['Year'] = anomalies['Date'].dt.year  # Extract the year

    # Calculate annual mean anomalies
    annual_mean = anomalies.groupby('Year')['Anomaly'].mean().reset_index()
    annual_mean['Region'] = region  # Add the region name
    
    # Append to the main DataFrame
    annual_means = pd.concat([annual_means, annual_mean], ignore_index=True)

A loop iterates through each region defined in the `regions` dictionary. For each region, a subset of the data (`region_data`) is extracted from the `annual_means` DataFrame, which contains yearly temperature anomalies for different regions. `local_min` and `local_max` are calculated to normalize the color scale for the selected region. For each region, a bar trace is then added to the figure with `go.Bar()`. Each trace is initially set as invisible (`visible=False`), meaning it won’t display until selected through the dropdown menu. The `marker` dictionary specifies that the color of the bars will vary based on the anomaly values, using a red-blue diverging color scale (`colorscale='RdBu_r'`). The cmin and cmax ensure that colors are appropriately mapped based on the local minimum and maximum anomaly values.

The first region's trace is set visible wen the figure is displayed.

The dropdown functionality is implemented in the layout update with the `updatemenus` parameter. 

In [ ]:
fig = go.Figure()

# Add traces for each region
for region in regions.keys():
    region_data = annual_means[annual_means['Region'] == region]
    local_min = region_data['Anomaly'].min()
    local_max = region_data['Anomaly'].max()
    
    fig.add_trace(go.Bar(
        x=region_data['Year'],
        y=region_data['Anomaly'],
        name=region,
        visible=False,  # Initially set as not visible
        marker=dict(
            color=region_data['Anomaly'],
            colorscale='RdBu_r',
            cmin=local_min,
            cmax=local_max,
            colorbar=dict(title='Anomaly (°C)')
        )
    ))

# Set the first region to be visible
fig.data[0].visible = True

# Update layout
fig.update_layout(
    title='Warming by Region',
    xaxis_title='Year',
    yaxis_title='Temperature Anomaly (°C)',
    template='plotly_white',
    title_x=0.5,
    updatemenus=[{ # Here the dropdown menu to switch between regions is created
        'buttons': [
            {
                'label': region,
                'method': 'update',
                'args': [{'visible': [region == r for r in regions.keys()]}],
            } for region in regions.keys()
        ],
        'direction': 'down',
        'showactive': True,
    }],
)

# Show the plot
fig.show()